In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
#Import basic libraries
%matplotlib inline


import os
import sys
import init
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from django_pandas.io import read_frame
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mpl_dates

from matplotlib.dates import date2num

import talib
from talib.abstract import *
from talib import MA_Type

from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest

/mnt/c/Users/anshu/Documents/experimental/env/lib/python3.8/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [3]:
from lib.pivots import getHHIndex, getHLIndex, getLLIndex, getLHIndex
#Create a strategy
class TrendFollow(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    order = 1
    
    sl = 0
    
    def init(self):
        # Precompute the two moving averages
        pass
    
    def next(self):
        hh_val = 0
        hl_val = 0
        lh_val = 0
        ll_val = 0
        d = self.data.df.index[-1].to_pydatetime()
        if (d.hour == 9 and d.minute==15) or (d.hour == 15 and d.minute>15):
            hh_val = 0
            hl_val = 0
            lh_val = 0
            ll_val = 0
            self.sl = 0
            return
        elif d.hour == 9 and d.minute<20:
            #No decision making in the first 5 minutes
            return
        elif d.hour==15 and d.minute==15:
            if self.position:
                #print('Close position')
                self.position.close()
                return
        hh = getHHIndex(self.data.Close, order=self.order)
        if hh is not None and len(hh)>1:
            hh = hh[-1]-self.order
            hh_val = self.data.Close[hh]
        else:
            hh = -1
        hl = getHLIndex(self.data.Close, order=self.order)
        if hl is not None and len(hl)>1:
            hl = hl[-1]-self.order
            hl_val = self.data.Close[hl]
        else:
            hl = -1
        ll = getLLIndex(self.data.Close, order=self.order)
        if ll is not None and len(ll)>1:
            ll = ll[-1]-self.order
            ll_val = self.data.Close[ll]
        else:
            ll = -1
        lh = getLHIndex(self.data.Close, order=self.order)
        if lh is not None and len(lh)>1:
            lh = lh[-1]-self.order
            lh_val = self.data.Close[lh]
        else:
            lh = -1
        
        #print(f'D: {self.data.df.index[-1]} HH:{hh} HL:{hl} LH:{lh} LL:{ll}')
        if hh>ll and hl>ll:
            #Colud be uptrend
            if hh > hl:
                #uptrend confirm
                if not self.position  and self.data.Close[-1] > hl_val:
                    #print('Long')
                    #print(self.data.df.tail(1))
                    self.sl = hl_val
                    #print(f'D: {self.data.df.index[-1]} O:{self.data.Open[-1]} H:{self.data.High[-1]} L:{self.data.Low[-1]} C:{self.data.Close[-1]} SL:{self.sl}')
                    self.buy(sl = self.sl)
                elif self.position.is_short  and self.data.Close[-1] > hl_val:
                    #print('Long')
                    #print(self.data.df.tail(1))
                    self.sl = hl_val
                    #print(f'D: {self.data.df.index[-1]} O:{self.data.Open[-1]} H:{self.data.High[-1]} L:{self.data.Low[-1]} C:{self.data.Close[-1]} SL:{self.sl}')
                    self.buy(sl = self.sl)
                else:
                    if self.sl != hl_val:
                        self.sl = hl_val
                        #print(f'D: {self.data.df.index[-1]} SL: {self.sl}')
                    #Else, update stop loss
            else:
                if hl > hh and self.data.Close[-1] > hh_val:
                    #uptrend pending confirm, but candle just closed above previous high
                    if not self.position and self.data.Close[-1] > hl_val:
                        #print('Long')
                        #print(self.data.df.tail(1))
                        self.sl = hl_val
                        #print(f'D: {self.data.df.index[-1]} O:{self.data.Open[-1]} H:{self.data.High[-1]} L:{self.data.Low[-1]} C:{self.data.Close[-1]} SL:{self.sl}')
                        self.buy(sl = self.sl)
                    elif self.position.is_short and self.data.Close[-1] > hl_val:
                        #print('Long')
                        #print(self.data.df.tail(1))
                        self.sl = hl_val
                        #print(f'D: {self.data.df.index[-1]} O:{self.data.Open[-1]} H:{self.data.High[-1]} L:{self.data.Low[-1]} C:{self.data.Close[-1]} SL:{self.sl}')
                        self.buy(sl = self.sl)
                    else:
                        if self.sl != hl_val:
                            self.sl = hl_val
                            #print(f'D: {self.data.df.index[-1]} SL: {self.sl}')
                        #Else, update stop loss
        if lh > hh and lh > hl:
            #downtrend possible
            if ll > lh:
                #downtrend confirm
                if not self.position and self.data.Close[-1] < lh_val:
                    #print('Short')
                    #print(self.data.df.tail(1))
                    self.sl = lh_val
                    #print(f'D: {self.data.df.index[-1]} O:{self.data.Open[-1]} H:{self.data.High[-1]} L:{self.data.Low[-1]} C:{self.data.Close[-1]} SL:{self.sl}')
                    self.sell(sl=self.sl)
                elif self.position.is_long  and self.data.Close[-1] < lh_val:
                    #print('Short')
                    #print(self.data.df.tail(1))
                    self.sl = lh_val
                    #print(f'D: {self.data.df.index[-1]} O:{self.data.Open[-1]} H:{self.data.High[-1]} L:{self.data.Low[-1]} C:{self.data.Close[-1]} SL:{self.sl}')
                    self.sell(sl=self.sl)
                else:
                    #Else, update stoploss
                    if self.position and self.sl != lh_val:
                        self.sl = lh_val
                        #print(f'D: {self.data.df.index[-1]} SL: {self.sl}')
            else:
                if lh > ll and self.data.Close[-1] < ll_val:
                    #downtrend confirm pending, but we're crossing over the lower low
                    if not self.position and self.data.Close[-1] < lh_val:
                        #print('Short')
                        #print(self.data.df.tail(1))
                        self.sl = lh_val
                        #print(f'D: {self.data.df.index[-1]} O:{self.data.Open[-1]} H:{self.data.High[-1]} L:{self.data.Low[-1]} C:{self.data.Close[-1]} SL:{self.sl}')
                        self.sell(sl=self.sl)
                    elif self.position.is_long  and self.data.Close[-1] < lh_val:
                        #print('Short')
                        #print(self.data.df.tail(1))
                        self.sl = lh_val
                        #print(f'D: {self.data.df.index[-1]} O:{self.data.Open[-1]} H:{self.data.High[-1]} L:{self.data.Low[-1]} C:{self.data.Close[-1]} SL:{self.sl}')
                        self.sell(sl=self.sl)
                    else:
                        #Else, update stoploss
                        if self.position and self.sl != lh_val:
                            self.sl = lh_val
                            #print(f'D: {self.data.df.index[-1]} SL: {self.sl}')

In [4]:
df = pd.read_csv('./NIFTY50_1m.csv')
#df = pd.read_csv('./NIFTY50_5m.csv')
#df = pd.read_csv('./NIFTY50_15m.csv')
#df = pd.read_csv('./NIFTY50_1h.csv')
#df = pd.read_csv('./NIFTY50_1d.csv')

for column in df.columns:
    if column != 'date':
       df[column] = pd.to_numeric(df[column])
#df['date'] = pd.to_datetime(df['date'])
#df['date'] = df['date'].tz_localize(None)
df.set_index('date', inplace=True)
df.index = pd.to_datetime(df.index)
df.index = df.index.tz_localize(None)

df = df.sort_index()
df = df.reindex(columns = ['open', 'high', 'low', 'close', 'volume'])
df.drop('volume', axis=1, inplace=True)
df.rename(columns={"open": "Open", "high": "High", "low": "Low", "close":"Close", "volume":'Volume'}, inplace=True)

df = df[~df.index.duplicated(keep='first')]
#Optionally, filter out by date range
#print(df.head(10))
filter = True
if filter:
    start_date = '2021-01-01 09:15:00'
    end_date = '2022-01-01 03:30:00'
    df = df.loc[start_date:end_date]


df.tail()

,Open,High,Low,Close
date,,,,
2021-12-31 15:25:00,17357.90,17358.50,17353.75,17356.30
2021-12-31 15:26:00,17356.10,17359.65,17355.55,17359.25
2021-12-31 15:27:00,17358.85,17360.50,17356.20,17359.80
2021-12-31 15:28:00,17359.75,17360.90,17351.10,17353.55
2021-12-31 15:29:00,17353.45,17365.45,17353.10,17364.25


In [5]:
bt = Backtest(df, TrendFollow, cash=100000, commission=0.03/100, trade_on_close=False, exclusive_orders = True)
stats = bt.run()
print(stats)

#bt.plot()

Start                     2021-01-01 09:15:00
End                       2021-12-31 15:29:00
Duration                    364 days 06:14:00
Exposure Time [%]                   71.369999
Equity Final [$]                  31417.07274
Equity Peak [$]                      100000.0
Return [%]                         -68.582927
Buy & Hold Return [%]               23.913967
Return (Ann.) [%]                  -68.891453
Volatility (Ann.) [%]                2.961844
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -68.591627
Avg. Drawdown [%]                  -68.591627
Max. Drawdown Duration      364 days 05:52:00
Avg. Drawdown Duration      364 days 05:52:00
# Trades                                 5236
Win Rate [%]                        15.508021
Best Trade [%]                       1.722396
Worst Trade [%]                     -0.740813
Avg. Trade [%]                    